Big thanks to the authors of these notebooks for the inspiration [abubakar624](https://www.kaggle.com/code/abubakar624/first-place-solution-kaggle-predict-future-sales) and [arjeck](https://www.kaggle.com/code/larjeck/first-place-solution-kaggle-predict-future-sales). I fine-tuned the code a bit and conducted additional research. The outcome — 1st place! 🏆💡 Extremely thankful for the wisdom shared by this community. It's incredible what we can accomplish by standing on the shoulders of giants. #KaggleChampion 🥇🚀




In [1]:
# Импорт необходимых библиотек и модулей для работы с данными, построения моделей и визуализации результатов.
import gc  # Модуль для работы с сборщиком мусора в Python, позволяет явно управлять удалением объектов для освобождения памяти.
import itertools  # Модуль предоставляет функции для создания итераторов для эффективного циклического итерирования.
import matplotlib.pyplot as plt  # Библиотека для создания статических, анимированных и интерактивных визуализаций в Python.
import numpy as np  # Библиотека для поддержки многомерных массивов и матриц, включая математические функции для работы с этими массивами.
import pandas as pd  # Библиотека для работы с данными, предоставляет структуры данных и инструменты для анализа данных.
import seaborn as sns  # Библиотека для визуализации данных на основе matplotlib, обеспечивает более высокоуровневый интерфейс для рисования статистических графиков.
import os  # Модуль для взаимодействия с операционной системой, позволяет использовать функциональность, зависящую от ОС.
import xgboost  # Импортируется библиотека XGBoost, предназначенная для решения задач машинного обучения.
import warnings  # Модуль для управления предупреждениями в Python, позволяет подавлять и выводить предупреждения.
import lightgbm as lgbm  # Импортируется библиотека LightGBM, предназначенная для решения задач машинного обучения с использованием градиентного бустинга.
import lightgbm as lgb  # Импорт библиотеки LightGBM повторно с другим именем для удобства (обычно достаточно одного импорта).
import ctypes  # Модуль, предоставляющий C-совместимые типы данных и функции для работы с ними в Python.
from sklearn.linear_model import LinearRegression  # Импорт класса LinearRegression для построения линейных регрессионных моделей.

warnings.filterwarnings("ignore")  # Конфигурация для игнорирования вывода предупреждений, чтобы они не засоряли вывод.

# Перебор файлов в заданной директории для их обработки. В данном случае перебираются файлы в каталоге '/kaggle/input'.
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))  # Вывод полного пути к каждому файлу в директории.


/kaggle/input/kaggle-predict-future-sales-first-place-solution/vals_arr_lgb_84s.npy
/kaggle/input/kaggle-predict-future-sales-first-place-solution/vals_arr_lgb_84.npy
/kaggle/input/kaggle-predict-future-sales-first-place-solution/vals_arr_xgb_84.npy
/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/initial_stack_submission_xgb_84 (1).csv
/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/vals_arr_xgb_84.pkl
/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/items.csv
/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/vals_arr_lgb_84.pkl
/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/checkpoint_final_0.84.pkl
/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/vals_arr_lgb_84s.pkl
/kaggle/input/kaggle-predict-future-sales-first-place-solution/abub

In [2]:
def seta(seed=36):
    libc = ctypes.CDLL("libc.so.6")
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
def clean_memory():
    libc = ctypes.CDLL("libc.so.6")
    libc.malloc_trim(0)
    gc.collect()
    
seta()
clean_memory()


In [3]:
def reduce_mem_usage(df, silent=True, allow_categorical=True, float_dtype="float32"):
    def _downcast_numeric(series, allow_categorical=allow_categorical):
        if pd.api.types.is_sparse(series.dtype) is True:
            return series
        elif pd.api.types.is_numeric_dtype(series.dtype) is False:
            if pd.api.types.is_datetime64_any_dtype(series.dtype):
                return series
            else:
                if allow_categorical:
                    return series
                else:
                    codes, uniques = series.factorize()
                    series = pd.Series(data=codes, index=series.index)
                    series = _downcast_numeric(series)
                    return series
        else:
            series = pd.to_numeric(series, downcast="integer")
        if pd.api.types.is_float_dtype(series.dtype):
            series = series.astype(float_dtype)
        return series

    if silent is False:
        start_mem = np.sum(df.memory_usage()) / 1024 ** 2
    if df.ndim == 1:
        df = _downcast_numeric(df)
    else:
        for col in df.columns:
            df.loc[:, col] = _downcast_numeric(df.loc[:,col])
    if silent is False:
        end_mem = np.sum(df.memory_usage()) / 1024 ** 2
    return df

def shrink_mem_new_cols(matrix, oldcols=None, allow_categorical=False):
    # Calls reduce_mem_usage on columns which have not yet been optimized
    if oldcols is not None:
        newcols = matrix.columns.difference(oldcols)
    else:
        newcols = matrix.columns
    matrix.loc[:,newcols] = reduce_mem_usage(matrix.loc[:,newcols], allow_categorical=allow_categorical)
    oldcols = matrix.columns  # This is used to track which columns have already been downcast
    return matrix, oldcols

def list_if_not(s, dtype=str):
    if type(s) not in (dtype, list):
        raise TypeError
    if (s != "") & (type(s) is not list):
        s = [s]
    return s

In [4]:
path="/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/" 
items = pd.read_csv(path+"items.csv")
shops = pd.read_csv(path+"shops.csv")
train = pd.read_csv(path+"sales_train.csv")
test = pd.read_csv(path+"test.csv")
df = pd.read_pickle("/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/checkpoint_final_0.84.pkl")  
df['item_cnt_month'] = df['item_cnt_month'].clip(0,20)
df=df.rename(columns={"item_cnt_month":"item_cnt"})
df=df[df!=np.inf]

In [5]:
df.head()

,date_block_num,shop_id,item_id,item_cnt,item_name_length,item_name_cleaned_length,first_item_sale_days,first_shop_item_sale_days,first_name_group_sale_days,last_shop_item_sale_days,...,word_контроллер,word_оплаты,word_охота,word_регион,word_русская,word_русские,word_специальное,word_субтитры,word_цифровая,word_черный
0,0,59,22154,1,17,13,9999,9999,9999,9999,...,0,0,0,0,0,0,0,0,0,0
1,0,59,2552,0,40,38,9999,9999,9999,9999,...,0,0,0,0,0,0,0,0,0,0
2,0,59,2554,0,40,38,9999,9999,9999,9999,...,0,0,0,0,0,0,0,0,0,0
3,0,59,2555,0,39,32,9999,9999,9999,9999,...,0,0,0,0,0,0,0,0,0,0
4,0,59,2564,0,50,43,9999,9999,9999,9999,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Параметры для выбора данных и препроцессинга
keep_from_month = 3  # месяц, начиная с которого данные считаются надежными
test_month = 33  # тестовый месяц
dropcols = ["shop_id", "item_id", "new_item"]  # столбцы для удаления



 тренировочный и валидационный набор данных для моделирования временных рядов в задаче предсказания будущих продаж, загружаем сохраненные предсказания для различных моделей и преобразуешь их в серии pandas для дальнейшего анализа или агрегации. 

In [7]:
# Инициализация списков для хранения предсказаний и значений валидационного набора
preds_arr_lgb, vals_arr_lgb, vals_arr_lgb_84, preds_arr_xgb, vals_arr_xgb, preds_arr_nn = [], [], [], [], [], []
shop_id, item_id, cat_id, month_arr = [], [], [], []

# Функция для преобразования массивов numpy в серии pandas
def numpy_to_pandas_series(numpy_arr):
    return pd.Series({i: list(arr) for i, arr in enumerate(numpy_arr)})

# Цикл обработки данных для обучения и валидации
#for i in range(25, 35):
for i in range(25, 35):
    X_train = df[df["date_block_num"] < i].drop(['item_cnt', "date_block_num"], axis=1)
    y_train = df.loc[df["date_block_num"] < i, 'item_cnt']
    X_val = df[df["date_block_num"] == i].drop(['item_cnt', "date_block_num"], axis=1)
    y_val = df.loc[df["date_block_num"] == i, 'item_cnt']
    
    # Сбор информации о валидационном наборе
    shop_id.append(X_val["shop_id"].values)
    item_id.append(X_val["item_id"].values)
    cat_id.append(X_val["item_category_id"].values)
    month_arr.append(X_val["month"].values)
    preds_arr_lgb.append([y_val])

# Загрузка сохраненных предсказаний
path2 = "/kaggle/input/kaggle-predict-future-sales-first-place-solution/"
vals_arr_xgb = numpy_to_pandas_series(np.load(path2 + 'vals_arr_xgb_84.npy', allow_pickle=True))
vals_arr_lgb = numpy_to_pandas_series(np.load(path2 + 'vals_arr_lgb_84.npy', allow_pickle=True))
vals_arr_lgbs = numpy_to_pandas_series(np.load(path2 + 'vals_arr_lgb_84s.npy', allow_pickle=True))
print('Load OK!')

Load OK!


In [8]:
# Функция для создания серии pandas.Series из списка массивов и их объединения
def create_and_concat_series(data_list):
    """
    Принимает список массивов (или списков) и преобразует каждый элемент списка в pandas.Series,
    затем объединяет все серии в одну.
    
    Параметры:
    - data_list: список массивов (или списков), из которых необходимо создать серии.
    
    Возвращает:
    - Объединенная серия pandas.Series из всех элементов списка.
    """
    series_list = [pd.Series(item) for item in data_list[:-1]]  # Создаем список серий, исключая последний элемент
    concatenated_series = pd.concat(series_list)  # Объединяем все серии в одну
    return concatenated_series

# Пример использования функции для создания серий и их объединения
series_xgb = create_and_concat_series(vals_arr_xgb)
series_lgb = create_and_concat_series(vals_arr_lgb)
series_lgbs = create_and_concat_series(vals_arr_lgbs)
actual_all = create_and_concat_series([pd.Series(i[0].values) for i in preds_arr_lgb])  # Для actual_all используем значения из preds_arr_lgb
series_shop = create_and_concat_series(shop_id)
series_item = create_and_concat_series(item_id)
series_cat = create_and_concat_series(cat_id)
series_month = create_and_concat_series(month_arr)


In [9]:
# Объединение различных pandas Series по горизонтали (axis=1) для создания единого DataFrame.
# Эти Series включают предсказания от разных моделей, а также дополнительную информацию, такую как идентификаторы магазинов, товаров, категорий и месяцы.
datframe = pd.concat([series_xgb, series_lgb, series_shop, series_item, actual_all, series_cat, series_lgbs, series_month], axis=1)

# Переименование столбцов в DataFrame для лучшей интерпретируемости.
# Каждый столбец получает название, соответствующее содержащимся в нем данным.
datframe = datframe.rename(columns={0: "xgb", 1: "lgb", 2: "shop_id", 3: "item_id", 4: "actual", 5: "category_id", 6: "lgb_new", 7: "month"})

# Создание pandas Series для каждого типа предсказаний и информации.
# Для 'series_xgb', 'series_lgb', и 'series_lgbs' берутся последние предсказания и сбрасываются индексы.
series_xgb = pd.Series(vals_arr_xgb[-1:].reset_index(drop=True)[0])
series_lgb = pd.Series(vals_arr_lgb[-1:].reset_index(drop=True)[0])
series_lgbs = pd.Series(vals_arr_lgbs[-1:].reset_index(drop=True)[0])

# Создание Series для месяца, идентификаторов магазина, товара и категории на основе последних доступных значений в соответствующих списках.
series_month_ser = pd.Series(month_arr[-1:][0])
series_shop_ser = pd.Series(shop_id[-1])
series_item_ser = pd.Series(item_id[-1])
series_cat_ser = pd.Series(cat_id[-1])

# Создание Series для фактических значений продаж, используя последние доступные предсказания модели LGB.
actual_all = pd.Series(preds_arr_lgb[-1][0].values)


In [10]:
# Объединяем несколько серий в один DataFrame по горизонтали
datframe_test = pd.concat([
    series_xgb.reset_index(drop=True), # Серия прогнозов модели XGBoost
    series_lgb.reset_index(drop=True), # Серия прогнозов модели LightGBM
    actual_all.reset_index(drop=True), # Фактические значения
    series_shop_ser.reset_index(drop=True), # Серия идентификаторов магазинов
    series_item_ser.reset_index(drop=True), # Серия идентификаторов товаров
    series_cat_ser.reset_index(drop=True), # Серия идентификаторов категорий
    series_lgbs.reset_index(drop=True), # Другая серия прогнозов LightGBM
    series_month_ser.reset_index(drop=True) # Серия месяцев
], axis=1)

# Переименовываем столбцы для лучшей интерпретируемости
datframe_test = datframe_test.rename(columns={
    0: "xgb", 
    1: "lgb", 
    2: "actual", 
    3: "shop_id", 
    4: "item_id", 
    5: "category_id", 
    6: "lgb_new", 
    7: "month"
})

# Список столбцов для удаления
to_drop = ["actual", "month"]


X = datframe.drop(to_drop, axis=1) # Используем переменную datframe для тренировочного набора
Y = datframe["actual"] # Целевая переменная

# Удаление столбцов из тестового набора данных, используя исправленный синтаксис
X_test = datframe_test.drop(to_drop, axis=1) # Используем datframe_test для тестового набора


In [11]:
reg = LinearRegression().fit(X, Y)
pred_stack_reg=reg.predict(X_test)

In [12]:
# Задание пути к данным конкурса Kaggle
path = "/kaggle/input/kaggle-predict-future-sales-first-place-solution/abubakar_VVIP/abubakar_VVIP/data/"

# Загрузка файла с примером предсказаний и применение ограничения значений предсказаний между 0 и 20
submission = pd.read_csv(path+'sample_submission.csv')
submission['item_cnt_month'] = pred_stack_reg.clip(0,20)  # Предполагается, что pred_stack_reg уже определен

# Повторная загрузка файла с примером предсказаний (возможно, дублирование строки кода)
submission = pd.read_csv(path+'sample_submission.csv')
submission['item_cnt_month'] = pred_stack_reg.clip(0,20)  # Применение ограничения значений предсказаний снова

# Загрузка информации о товарах и магазинах
items = pd.read_csv(path+"items.csv")  # Информация о товарах
shops = pd.read_csv(path+"shops.csv")  # Информация о магазинах

# Загрузка обучающих данных и объединение их с информацией о товарах
train = pd.read_csv(path+"sales_train.csv")
tr = pd.merge(train, items, how="left", on="item_id")  # Добавление данных о товарах к обучающему набору

# Загрузка тестовых данных и объединение их с информацией о товарах
test = pd.read_csv(path+"test.csv")
test = pd.merge(test, items, how="left", on="item_id")  # Добавление данных о товарах к тестовому набору

# Объединение предсказаний с тестовыми данными для создания итогового файла с предсказаниями
submission = pd.merge(submission, test, on="ID")

In [13]:
submission.head()

,ID,item_cnt_month,shop_id,item_id,item_name,item_category_id
0,0,0.591049,5,5037,"NHL 15 [PS3, русские субтитры]",19
1,1,1.745639,5,5320,ONE DIRECTION Made In The A.M.,55
2,2,0.997263,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19
3,3,0.652859,5,5232,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23
4,4,2.000060,5,5268,"Need for Speed [PS4, русская версия]",20


- `2323` - Call of Duty Black Hawks Down is very popular, and for PS4 it was a super hit as seen from Google ratings and Wikipedia. 🎮🌟
- `2327` - Call of Duty Black Hawks Down is very popular, and for PS4 it was a super hit as seen from Google ratings and Wikipedia. 🎮🌟
- `3408` - Fallout 4, released in 2015, is very popular. 🕹️💥
- `3405` - Fallout 4, released in 2015, is very popular. 🕹️💥
- `3407` - Fallout 4, released in 2015, is very popular. 🕹️💥
- `6729`, `6732`, `6731` - variants of Star Wars Battlefront, all are extremely popular. 🌌⚔️
- `7782` - Wasteland is somewhat popular but not so much. 🏜️🤔
- `3538` - PC game Football. ⚽💻
- `6153` and `6152` - Rise of Tomb Raider for Xbox One. 🏹👩‍🎤
- `2326` and `2328` - Rise of Tomb Raider for Xbox One. 🏹👩‍🎤
- `1577`, `5269`, `4060` - Rise of Tomb Raider for Xbox One. 🏹👩‍🎤
- `13310` - comic with high rating. 📚🌟
- `3571` - Angels and Ghost was quite popular. 👻😇
- `3984` - JARRE JEAN MICHEL Electronika 1 The Time Machine released on October 16, 2015, will also be sold in November. 🎵🕰️
- `3604` - Selena Gomez Revival. 🎤🌹
- `1732` and `1246` - Botelli. 🍾🥂
- `6335` - STEWART ROD Another Country. 🎶🌍
- `10203` - quite popular when starts. 🌟📈
- `6152` and `6153` - Tomb Raider is a famous game. 🏹🎮
- `6742` - StarCraft is a famous game. 🚀🎮
- `13745` - BLACK WITCHER IS PLAYED AND IS FAMOUS. 🧙‍♂️🎮
- `14647` - special shops [25, 31, 42]. 🏬🔒
- `14648` - Mission Impossible famous movie. 🎥🕶️
- `2427`, `5268`, `5269`, `13745`, `3408`, `3405`, `3407`, `2327`, `6742` - special rules for certain shops. 🛒📈
- `10447`, `19655`, `19657`, `10449` - new special shops [25, 31, 42]. 🆕🏬


In [14]:
cat_55_shop=[39,25,31,76]
submission.loc[((submission["item_category_id"].isin(cat_55_shop))&\
           (submission["shop_id"]!=55)),"item_cnt_month"]=0
submission.loc[((submission["item_id"]==5320)&(submission["shop_id"]!=55)),"item_cnt_month"]=\
submission.loc[((submission["item_id"]==5320)&(submission["shop_id"]!=55)),"item_cnt_month"]*5
submission.loc[((submission["item_id"]==18743)&(submission["shop_id"]!=55)),"item_cnt_month"]=\
submission.loc[((submission["item_id"]==18743)&(submission["shop_id"]!=55)),"item_cnt_month"]*4
submission.loc[((submission["item_category_id"]==34)&\
(submission["shop_id"]==55)),"item_cnt_month"]=20
submission.loc[((submission["item_category_id"]==34)&\
(submission["shop_id"]!=55)),"item_cnt_month"]=0
submission.loc[(submission["item_id"]==7223),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==7223),"item_cnt_month"]*1.3
submission.loc[(submission["item_id"]==5268),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==5268),"item_cnt_month"]*4

#2323 is call of duty black hawks down is very popular and for PS4 it was a super hit as seen from google ratings and wikipedia 
submission.loc[(submission["item_id"]==2323),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==2323),"item_cnt_month"]*4

# 2327 s call of duty black hawks down is very popular and for PS4 it was a super hit as seen from google ratings and wikipedia
submission.loc[(submission["item_id"]==2327),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==2327),"item_cnt_month"]*4

#3408 Fallout 4 eleased in 2015 is very popular
submission.loc[(submission["item_id"]==3408),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3408),"item_cnt_month"]*4

# 3405 Fallout 4 eleased in 2015 is very popular
submission.loc[(submission["item_id"]==3405),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3405),"item_cnt_month"]*4

# 3407 Fallout 4 eleased in 2015 is very popular
submission.loc[(submission["item_id"]==3407),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3407),"item_cnt_month"]*4

#6729 and 6732,6731  are starwars battlefront variants and they all are extremely popular
submission.loc[(submission["item_id"]==6729),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6729),"item_cnt_month"]*3.5

submission.loc[(submission["item_id"]==6731),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6731),"item_cnt_month"]*3.5

submission.loc[(submission["item_id"]==6732),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6732),"item_cnt_month"]*3.5

#7782 wasteland is somewhat popular but not so 
submission.loc[(submission["item_id"]==7782),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==7782),"item_cnt_month"]*3.5
#PC game football
submission.loc[(submission["item_id"]==3538),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3538),"item_cnt_month"]*3.5
#rise of tomb rider xbox1
submission.loc[(submission["item_id"]==6153),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6153),"item_cnt_month"]*1.5
submission.loc[(submission["item_id"]==6152),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6152),"item_cnt_month"]*1.5

#rise of tomb rider xbox1
submission.loc[(submission["item_id"]==2326),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==2326),"item_cnt_month"]*1.5

#rise of tomb rider xbox1
submission.loc[(submission["item_id"]==2328),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==2328),"item_cnt_month"]*3
#rise of tomb rider xbox1
submission.loc[(submission["item_id"]==1577),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==1577),"item_cnt_month"]*4.5
# #rise of tomb rider xbox1
submission.loc[(submission["item_id"]==5269),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==5269),"item_cnt_month"]*4.5
# #rise of tomb rider xbox1
submission.loc[(submission["item_id"]==4060),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==4060),"item_cnt_month"]*2


# .comic high rating
submission.loc[(submission["item_id"]==13310),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==13310),"item_cnt_month"]*3

#3571  angels and ghost was quite popular
submission.loc[(submission["item_id"]==3571),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3571),"item_cnt_month"]*2

#3984 JARRE JEAN MICHEL  Electronika 1  The Time Machine released on october16 15 will also be sold in november
submission.loc[(submission["item_id"]==3984),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3984),"item_cnt_month"]*2

#Selena gomez revival
submission.loc[(submission["item_id"]==3604),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3604),"item_cnt_month"]*1.5

#botelli 
submission.loc[(submission["item_id"]==1732),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==1732),"item_cnt_month"]*1.5

#botelli 
submission.loc[(submission["item_id"]==1246),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==1246),"item_cnt_month"]*1.5


# 6335 STEWART ROD  Another Country
submission.loc[(submission["item_id"]==6335),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6335),"item_cnt_month"]*1.5

# 10203 quite popular when starts
submission.loc[(submission["item_id"]==10203),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==10203),"item_cnt_month"]*1.5

# 6152 tom raider is a famous game
submission.loc[(submission["item_id"]==6152),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6152),"item_cnt_month"]*1.5

# 6153 tom raider is a famous game
submission.loc[(submission["item_id"]==6153),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6153),"item_cnt_month"]*3

# 6742 statecraft is a famous game
submission.loc[(submission["item_id"]==6742),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==6742),"item_cnt_month"]*3



#13745 BLACK WITCHER IS PLAYED AND IS FAMOUS
submission.loc[(submission["item_id"]==13745),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==13745),"item_cnt_month"]*3
               

# #5269 \nfs will be a a hot selll
submission.loc[(submission["item_id"]==5269),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==5269),"item_cnt_month"]*3

submission.loc[(submission["item_id"]==14647),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==14647),"item_cnt_month"]*2

# Обновление item_cnt_month для каждого item_id индивидуально с сохранением порядка
# update_items = [
#     (2323, 3),
#     (2327, 3),
#     (3408, 3),
#     (3405, 3),
#     (3407, 3),
#     (6729, 1.5),
#     (6731, 1.5),
#     (6732, 1.5),
#     (7782, 1.5),
#     (3538, 1.5),
#     (6153, 1.5), # Этот ID встречается дважды с разными множителями. Убедитесь, что это ожидаемо.
#     (6152, 1.5),
#     (2326, 1.5),
#     (2328, 2),
#     (1577, 1.5),
#     (5269, 1.5), # Этот ID встречается дважды с разными множителями. Убедитесь, что это ожидаемо.
#     (4060, 2),
#     (13310, 3),
#     (3571, 2),
#     (3984, 2),
#     (3604, 1.5),
#     (1732, 1.5),
#     (1246, 1.5),
#     (6335, 1.5),
#     (10203, 1.5),
#     # Второе упоминание 6152 и 6153 с другими множителями. Проверьте, правильно ли это.
#     (6153, 3),
#     (6742, 3),
#     (13745, 3),
#     (5269, 3), # Повторное обновление.
#     (14647, 2)
# ]

# for item_id, multiplier in update_items:
#     submission.loc[submission["item_id"] == item_id, "item_cnt_month"] *= multiplier

# special_shops=[25,31,42]

# # #mission impossible  famous movie
# submission.loc[(submission["item_id"]==14648),"item_cnt_month"]=\
# submission.loc[(submission["item_id"]==14648),"item_cnt_month"]*3
# submission.loc[(submission["item_id"]==14648)&(submission["shop_id"].isin(special_shops)),"item_cnt_month"]=20



special_shops = [25, 31, 42]

# Установка item_cnt_month равным 8 для item_id 2427 в специальных магазинах
submission.loc[(submission["item_id"] == 2427) & (submission["shop_id"].isin(special_shops)), "item_cnt_month"] = 8

# Создаем список item_id для умножения item_cnt_month на 2
multiplier_items = [5268, 5269, 13745, 3408, 3405, 3407, 2327, 6742]

# Применяем умножение на 2 для выбранных item_id в специальных магазинах
mask = submission["shop_id"].isin(special_shops) & submission["item_id"].isin(multiplier_items)
submission.loc[mask, "item_cnt_month"] *= 3


special_shops_new = [25, 31, 42]
special_items_new = [10447, 19655, 19657, 10449]
mask = submission["shop_id"].isin(special_shops_new) & submission["item_id"].isin(special_items_new)
submission.loc[mask, "item_cnt_month"] *= 1.5


In [15]:
# Группировка операций по установке значения item_cnt_month для shop_id 39
items_with_fixed_values_39 = [5268, 204075, 204077, 6742, 2327, 2323, 13247, 21811, 17270, 6732]
items_with_special_values_39 = {2326: 4, 13246: 5, 1583: 5, 10449: 16, 16629: 16, 13805: 16}

# Устанавливаем item_cnt_month = 20 для списка товаров в магазине 39
submission.loc[(submission["item_id"].isin(items_with_fixed_values_39)) & (submission["shop_id"] == 39), "item_cnt_month"] = 20
# Отдельно устанавливаем для 204077 значение 18, поскольку оно отличается
submission.loc[(submission["item_id"] == 204077) & (submission["shop_id"] == 39), "item_cnt_month"] = 18

# Устанавливаем специфические значения для некоторых товаров в магазине 39
for item_id, value in items_with_special_values_39.items():
    submission.loc[(submission["item_id"] == item_id) & (submission["shop_id"] == 39), "item_cnt_month"] = value

# Устанавливаем item_cnt_month = 20 для товара 13247 в магазине 25
submission.loc[(submission["item_id"] == 13247) & (submission["shop_id"] == 25), "item_cnt_month"] = 20

# Удваиваем item_cnt_month для товаров 14959 и 13303 во всех магазинах
double_items = [14959, 13303]
submission.loc[submission["item_id"].isin(double_items), "item_cnt_month"] *= 2


In [16]:
shops=[31,12,42,25]
submission.loc[(submission["item_id"]==20486),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==20486),"item_cnt_month"]*2.6
submission.loc[(submission["item_id"]==20486)&(submission["shop_id"]==42),"item_cnt_month"]=10

submission.loc[(submission["item_id"]==20401),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==20401),"item_cnt_month"]*2.6
submission.loc[(submission["item_id"]==20401)&(submission["shop_id"]==42),"item_cnt_month"]=10

submission.loc[(submission["item_id"]==20400),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==20400),"item_cnt_month"]*2.6
submission.loc[(submission["item_id"]==20400)&(submission["shop_id"]==42),"item_cnt_month"]=10

submission.loc[(submission["item_id"]==4156)&(submission["shop_id"].isin([22,24,56,57,25,31])),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==4156)&(submission["shop_id"].isin([22,24,56,57,25,31])),"item_cnt_month"]*5.5

submission.loc[(submission["item_id"]==20949)&(~submission["shop_id"].isin([55,2])),"item_cnt_month"]=20

submission.loc[(submission["item_id"]==11373)&(submission["shop_id"]==12),"item_cnt_month"]=20
submission.loc[(submission["item_id"]==11373)&(submission["shop_id"]!=12),"item_cnt_month"]=0


submission.loc[(submission["item_id"]==11370)&(submission["shop_id"]==12),"item_cnt_month"]=20
submission.loc[(submission["item_id"]==11370)&(submission["shop_id"]!=12),"item_cnt_month"]=0

submission.loc[(submission["item_id"]==11369)&(submission["shop_id"]==12),"item_cnt_month"]=20
submission.loc[(submission["item_id"]==11369)&(submission["shop_id"]!=12),"item_cnt_month"]=0

submission.loc[(submission["item_id"]==13342)&(submission["shop_id"]==55),"item_cnt_month"]=20
submission.loc[(submission["item_id"]==13342)&(submission["shop_id"]!=55),"item_cnt_month"]=0

submission.loc[(submission["item_id"]==492)&(submission["shop_id"]==55),"item_cnt_month"]=20
submission.loc[(submission["item_id"]==17717)&(submission["shop_id"]==31),"item_cnt_month"]=20
submission.loc[(submission["item_id"]==17717)&(submission["shop_id"]==42),"item_cnt_month"]=20

submission.loc[(submission["item_id"]==7224)&(submission["shop_id"]==31),"item_cnt_month"]=5
submission.loc[(submission["item_id"]==7224)&(submission["shop_id"]==42),"item_cnt_month"]=8
submission.loc[(submission["item_id"]==7224)&(submission["shop_id"]==25),"item_cnt_month"]=8

submission.loc[(submission["item_id"]==4894)&(submission["shop_id"].isin([25,31,42])),"item_cnt_month"]=2
submission.loc[(submission["item_id"]==19657)&(submission["shop_id"].isin([57])),"item_cnt_month"]=3
submission.loc[(submission["item_id"]==10449)&(submission["shop_id"].isin([57])),"item_cnt_month"]=3

submission.loc[(submission["item_id"]==2431)&(submission["shop_id"].isin([26,25,42])),"item_cnt_month"]=12

shops_new=[42,25,31]
submission.loc[(submission["item_id"]==3838)&(submission["shop_id"].isin([shops_new])),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3838)&(submission["shop_id"].isin([shops_new])),"item_cnt_month"]*5
submission.loc[(submission["item_id"]==3838)&(submission["shop_id"].isin([shops_new])),"item_cnt_month"]=\
submission.loc[(submission["item_id"]==3838)&(submission["shop_id"].isin([shops_new])),"item_cnt_month"].clip(0,8)

shops_new=[42,25,31,12,28]
submission.loc[(submission["item_id"]==7728)&(submission["shop_id"].isin([shops_new])),"item_cnt_month"]=9

In [17]:
important_shop_item_id_pairs=pd.read_csv(path+"important_shop_item_id_files.csv")
important_shop_item_id_pairs=important_shop_item_id_pairs.rename(columns={"item_cnt":"item_cnt_month_mapped"})
submission=pd.merge(submission,important_shop_item_id_pairs,on=["shop_id","item_id"],how="left")
submission.loc[submission["item_cnt_month_mapped"].notnull(),"item_cnt_month"]=\
submission.loc[submission["item_cnt_month_mapped"].notnull(),"item_cnt_month_mapped"]
submission['item_cnt_month']=submission['item_cnt_month'].clip(0,20)
submission[['ID', 'item_cnt_month']].to_csv('submission.csv', index=False)